# PROCESO DE EXTRACCIÓN - TRANSFORMACIÓN - LIMPIEZA DE LOS REGISTROS DE ACCIDENTES OCURRIDOS EN LA CIUDAD DE BUENOS AIRES

## 1 - DESCARGA DE LOS DATASETS PARA EL PROCESO...

url = https://data.buenosaires.gob.ar/dataset/victimas-siniestros-viales

## 2 - IMPORTACIÓN DE LAS LIBRERIAS PARA EL PROCESO...

In [2]:
import os
import pandas as pd
from datetime import datetime, time
from datetime import datetime, time, timedelta
import numpy as np

## 3 - EXPLORACIÓN INICIAL DE LOS ARCHIVOS CON FORMATO .xlsx Y CREACIÓN DE LOS DATAFRAMES PARA EL PROCESO DE ETL...

In [3]:
# LECTURA DE LOS ARCHIVOS...

# Ruta de la carpeta que contiene los archivos
carpeta_dataset = "DATASET_BRUTO"

# Nombre de los archivos
archivo_homicidios = "homicidios.xlsx"
archivo_lesiones = "lesiones.xlsx"

# Ruta completa de los archivos
ruta_homicidios = os.path.join(carpeta_dataset, archivo_homicidios)
ruta_lesiones = os.path.join(carpeta_dataset, archivo_lesiones)

# Obtener nombres de las hojas en el archivo 'homicidios.xlsx'
hojas_homicidios = pd.ExcelFile(ruta_homicidios).sheet_names

# Obtener nombres de las hojas en el archivo 'lesiones.xlsx'
hojas_lesiones = pd.ExcelFile(ruta_lesiones).sheet_names

# Imprimir en pantalla los nombres de las hojas
print("Hojas en el archivo 'homicidios.xlsx':")
print(hojas_homicidios)

print("\nHojas en el archivo 'lesiones.xlsx':")
print(hojas_lesiones)

Hojas en el archivo 'homicidios.xlsx':
['HECHOS', 'DICCIONARIO_HECHOS', 'VICTIMAS', 'DICCIONARIO_VICTIMAS', 'clas']

Hojas en el archivo 'lesiones.xlsx':
['HECHOS', 'DICCIONARIO_HECHOS', 'VICTIMAS', 'DICCIONARIO_VICTIMAS']


In [4]:
# CREACIÓN DE LOS DATAFRAMES CORRESPONDIENTES AL ARCHIVO "lesiones.xlsx"

# Ruta del archivo
archivo_excel = 'DATASET_BRUTO/lesiones.xlsx'

# Lee la hoja 'HECHOS' del archivo Excel en un DataFrame
try:
    hechos_lesiones = pd.read_excel(archivo_excel, sheet_name='HECHOS')
    diccionario_hechos_lesiones = pd.read_excel(archivo_excel, sheet_name='DICCIONARIO_HECHOS')
    victimas_lesiones = pd.read_excel(archivo_excel, sheet_name='VICTIMAS')
    diccionario_victimas_lesiones = pd.read_excel(archivo_excel, sheet_name='DICCIONARIO_VICTIMAS')
except Exception as e:
    print(f"Error al leer el archivo: {e}")

print("EL PROCESO EN EL ARCHIVO lesiones.xlsx SE EJECUTÓ CON ÉXITO...")

EL PROCESO EN EL ARCHIVO lesiones.xlsx SE EJECUTÓ CON ÉXITO...


## 4 - PROCESO DE ETL PARA EL ARCHIVO lesiones.xlsx...

En esta parte del proceso se tiene disponible los siguientes DATAFRAMES que resultaron del archivo "homicidios.xlsx". Los dataframes mencionados son:

* hechos_lesiones
* diccionario_hechos_lesiones
* victimas_lesiones
* diccionario_victimas_lesiones

De estos dataframes, solo se utilizará el denominado "hechos_lesiones" y "victimas_lesiones".

### a. DATAFRAME "HECHOS_LESIONES"...

#### (1) EXPLORACIÓN INICIAL...

In [5]:
hechos_lesiones.head(5)

,id,n_victimas,aaaa,mm,dd,fecha,hora,franja_hora,direccion_normalizada,comuna,...,latutid,victima,acusado,participantes,moto,auto,transporte_publico,camion,ciclista,gravedad
0,LC-2019-0000179,1,2019,1,1,2019-01-01 00:00:00,09:00:00,9,SD,14,...,-34.559658,CICLISTA,SD,CICLISTA-SD,SD,SD,SD,SD,x,SD
1,LC-2019-0000053,1,2019,1,1,2019-01-01 00:00:00,01:55:00,1,SD,8,...,-34.669125,AUTO,SD,AUTO-SD,SD,x,SD,SD,SD,SD
2,LC-2019-0000063,1,2019,1,1,2019-01-01 00:00:00,02:00:00,2,SD,8,...,-34.677556,SD,SD,SD-SD,SD,SD,SD,SD,SD,SD
3,LC-2019-0000079,1,2019,1,1,2019-01-01 00:00:00,02:30:00,2,SD,7,...,-34.647349,PEATON,SD,PEATON-SD,x,SD,SD,SD,SD,SD
4,LC-2019-0000082,4,2019,1,1,2019-01-01 00:00:00,04:30:00,4,SD,3,...,-34.604579,AUTO,SD,AUTO-SD,SD,SD,x,SD,SD,SD


In [6]:
hechos_lesiones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23785 entries, 0 to 23784
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     23785 non-null  object 
 1   n_victimas             23785 non-null  int64  
 2   aaaa                   23785 non-null  int64  
 3   mm                     23785 non-null  int64  
 4   dd                     23785 non-null  int64  
 5   fecha                  23785 non-null  object 
 6   hora                   23785 non-null  object 
 7   franja_hora            23780 non-null  object 
 8   direccion_normalizada  23732 non-null  object 
 9   comuna                 23616 non-null  object 
 10  tipo_calle             23785 non-null  object 
 11  otra_direccion         23785 non-null  object 
 12  calle                  12867 non-null  object 
 13  altura                 12771 non-null  float64
 14  cruce                  9407 non-null   object 
 15  ge

In [7]:
hechos_lesiones.dtypes

id                        object
n_victimas                 int64
aaaa                       int64
mm                         int64
dd                         int64
fecha                     object
hora                      object
franja_hora               object
direccion_normalizada     object
comuna                    object
tipo_calle                object
otra_direccion            object
calle                     object
altura                   float64
cruce                     object
geocodificacion_CABA      object
longitud                  object
latutid                   object
victima                   object
acusado                   object
participantes             object
moto                      object
auto                      object
transporte_publico        object
camion                    object
ciclista                  object
gravedad                  object
dtype: object

In [8]:
hechos_lesiones.shape

(23785, 27)

In [9]:
hechos_lesiones.isnull().sum()

id                           0
n_victimas                   0
aaaa                         0
mm                           0
dd                           0
fecha                        0
hora                         0
franja_hora                  5
direccion_normalizada       53
comuna                     169
tipo_calle                   0
otra_direccion               0
calle                    10918
altura                   11014
cruce                    14378
geocodificacion_CABA        39
longitud                   262
latutid                    262
victima                      0
acusado                      0
participantes                0
moto                        93
auto                        93
transporte_publico          93
camion                      93
ciclista                    93
gravedad                     0
dtype: int64

In [10]:
hechos_lesiones.columns

Index(['id', 'n_victimas', 'aaaa', 'mm', 'dd', 'fecha', 'hora', 'franja_hora',
       'direccion_normalizada', 'comuna', 'tipo_calle', 'otra_direccion',
       'calle', 'altura', 'cruce', 'geocodificacion_CABA', 'longitud',
       'latutid', 'victima', 'acusado', 'participantes', 'moto', 'auto',
       'transporte_publico', 'camion', 'ciclista', 'gravedad'],
      dtype='object')

#### (2) PROCESO DE ETL DEL DATAFRAME...

In [11]:
# ELIMINACIÓN DE LAS COLUMNAS INNECESARIAS...

columnas_a_eliminar = ['fecha', 'hora', 'calle', 'altura', 'cruce', 'direccion_normalizada', 'otra_direccion', 'participantes', 'geocodificacion_CABA', 'tipo_calle']
hechos_lesiones = hechos_lesiones.drop(columnas_a_eliminar, axis=1)

In [12]:
# RENOMBRAR LAS COLUMNAS...

nuevos_nombres = {'id': 'ID', 'n_victimas':'NUMERO_DE_VICTIMAS', 'aaaa': 'AÑO', 'mm': 'MES', 'dd': 'DIA', 'franja_hora': 'FRANJA_HORARIA', 'comuna': 'COMUNA', 'longitud': 'LONGITUD', 'latutid': 'LATITUD', 'gravedad': 'GRAVEDAD', 'victima': 'VICTIMA', 'acusado': 'ACUSADO'}
hechos_lesiones = hechos_lesiones.rename(columns=nuevos_nombres)

In [13]:
# TRABAJO CON LAS COLUMNAS DE LOS VEHICULOS DEL DATAFRAME...

# Preparación de la columna moto
valor_a_cambiar_moto = ['x', '0']
nuevo_valor_moto = 'moto'
hechos_lesiones['moto'] = hechos_lesiones['moto'].replace(valor_a_cambiar_moto, nuevo_valor_moto)

# Preparación de la columna auto
valor_a_cambiar_auto = ['x', '0']
nuevo_valor_auto = 'auto'
hechos_lesiones['auto'] = hechos_lesiones['auto'].replace(valor_a_cambiar_auto, nuevo_valor_auto)

# Preparación de la columna transporte_publico
valor_a_cambiar_transporte_publico = ['x', '0']
nuevo_valor_transporte_publico = 'transporte_publico'
hechos_lesiones['transporte_publico'] = hechos_lesiones['transporte_publico'].replace(valor_a_cambiar_transporte_publico, nuevo_valor_transporte_publico)

# Preparación de la columna camion
valor_a_cambiar_camion = ['x', '0']
nuevo_valor_camion = 'camion'
hechos_lesiones['camion'] = hechos_lesiones['camion'].replace(valor_a_cambiar_camion, nuevo_valor_camion)

# Preparación de la columna ciclista
valor_a_cambiar_ciclista = ['x', '0']
nuevo_valor_ciclista = 'ciclista'
hechos_lesiones['ciclista'] = hechos_lesiones['ciclista'].replace(valor_a_cambiar_ciclista, nuevo_valor_ciclista)

# Crear la nueva columna 'VEHÍCULO_HECHO'
hechos_lesiones['VEHICULO_SINIESTRO'] = hechos_lesiones.apply(lambda row: 'moto' if row['moto'] == 'moto'
                                          else 'auto' if row['auto'] == 'auto'
                                          else 'transporte_publico' if row['transporte_publico'] == 'transporte_publico'
                                          else 'camion' if row['camion'] == 'camion'
                                          else 'ciclista' if row['ciclista'] == 'ciclista'
                                          else None, axis=1)

# Lista de columnas que deseas eliminar
columnas_a_eliminar = ['moto', 'auto', 'transporte_publico', 'camion', 'ciclista']

# Eliminar las columnas especificadas
hechos_lesiones = hechos_lesiones.drop(columnas_a_eliminar, axis=1)

In [14]:
# TRABAJO CON LA COLUMNA "FRANJA_HORARIA"

hechos_lesiones['FRANJA_HORARIA'] = hechos_lesiones['FRANJA_HORARIA'].replace(['sd', np.nan], 17)

hechos_lesiones['FRANJA_HORARIA'] = hechos_lesiones['FRANJA_HORARIA'].astype(int)

In [15]:
# TRABAJO CON LA COLUMNA "COMUNA"...

# Eliminar filas con NaN en las columnas especificadas
columnas_a_verificar = ['COMUNA']
hechos_lesiones = hechos_lesiones.dropna(subset=columnas_a_verificar)

# Eliminar filas con 'SD', 'sd' y None en las columnas especificadas
valores_a_eliminar = ['SD', 'sd', 'No Especificada', None, np.nan]
for columna in columnas_a_verificar:
    hechos_lesiones = hechos_lesiones[~hechos_lesiones[columna].isin(valores_a_eliminar)]

hechos_lesiones['COMUNA'] = hechos_lesiones['COMUNA'].astype(int)

In [16]:
# TRABAJO CON LA COLUMNA "LONGITUD" Y "LATITUD"...

# Eliminar filas con NaN en las columnas especificadas
columnas_a_verificar = ['LONGITUD', 'LATITUD']
hechos_lesiones = hechos_lesiones.dropna(subset=columnas_a_verificar)

# Eliminar filas con 'SD', 'sd' y None en las columnas especificadas
valores_a_eliminar = ['SD', 'sd', None, np.nan]
for columna in columnas_a_verificar:
    hechos_lesiones = hechos_lesiones[~hechos_lesiones[columna].isin(valores_a_eliminar)]

In [17]:
# TRABAJO CON LA COLUMNA "VICTIMA"

hechos_lesiones['VICTIMA'] = hechos_lesiones['VICTIMA'].replace('SD', 'MOTO')

In [18]:
# TRABAJO CON LA COLUMNA "ACUSADO"

hechos_lesiones['ACUSADO'] = hechos_lesiones['ACUSADO'].replace('SD', 'AUTO')

In [19]:
# TRABAJO CON LA COLUMNA "VEHICULO_SINIESTRO"

hechos_lesiones['VEHICULO_SINIESTRO'] = hechos_lesiones['VEHICULO_SINIESTRO'].fillna('auto')

In [20]:
# TRABAJO CON LA COLUMNA "GRAVEDAD"

hechos_lesiones['GRAVEDAD'] = hechos_lesiones['GRAVEDAD'].replace(['sd', 'SD'], 'GRAVE')

In [21]:
# NORMALIZACIÓN DE LOS DATOS DE LAS COLUMNAS EN UN SOLO TIPO DE DATOS...

hechos_lesiones['ID'] = hechos_lesiones['ID'].astype(str)
hechos_lesiones['VICTIMA'] = hechos_lesiones['VICTIMA'].astype(str)
hechos_lesiones['ACUSADO'] = hechos_lesiones['ACUSADO'].astype(str)
hechos_lesiones['GRAVEDAD'] = hechos_lesiones['GRAVEDAD'].astype(str)
hechos_lesiones['VEHICULO_SINIESTRO'] = hechos_lesiones['VEHICULO_SINIESTRO'].astype(str)

In [22]:
# TRABAJO CON LA COLUMNA "pos x" Y "pos y"

hechos_lesiones['LONGITUD'] = pd.to_numeric(hechos_lesiones['LONGITUD'], errors='coerce')
hechos_lesiones['LATITUD'] = pd.to_numeric(hechos_lesiones['LATITUD'], errors='coerce')

In [23]:
# TRATAMIENTO DE VALORES DEL DATAFRAME...

# Identificar valores nulos en el DataFrame y contarlos por columna
valores_nulos = hechos_lesiones.isnull().sum()

# Identificar valores faltantes (en blanco) en el DataFrame y contarlos por columna
valores_faltantes = (hechos_lesiones == '').sum()

# Identificar valores iguales a "SD" en el DataFrame y contarlos por columna
valores_SD = (hechos_lesiones == 'SD').sum()

# Identificar valores iguales a "sd" en el DataFrame y contarlos por columna
valores_sd = (hechos_lesiones == 'sd').sum()

# Calcular porcentajes
porcentaje_nulos = (valores_nulos / len(hechos_lesiones)) * 100
porcentaje_faltantes = (valores_faltantes / len(hechos_lesiones)) * 100
porcentaje_SD = (valores_SD / len(hechos_lesiones)) * 100
porcentaje_sd = (valores_sd / len(hechos_lesiones)) * 100

# Crear un DataFrame con los resultados
resultados_datos_hechos_lesiones = pd.DataFrame({
    'Valores Nulos': valores_nulos,
    'Porcentaje Nulos': porcentaje_nulos,
    'Valores Faltantes': valores_faltantes,
    'Porcentaje Faltantes': porcentaje_faltantes,
    'Valores "SD"': valores_SD,
    'Porcentaje "SD"': porcentaje_SD,
    'Valores "sd"': valores_sd,
    'Porcentaje "sd"': porcentaje_sd
})

In [24]:
resultados_datos_hechos_lesiones

,Valores Nulos,Porcentaje Nulos,Valores Faltantes,Porcentaje Faltantes,"Valores ""SD""","Porcentaje ""SD""","Valores ""sd""","Porcentaje ""sd"""
ID,0,0.0,0,0.0,0,0.0,0,0.0
NUMERO_DE_VICTIMAS,0,0.0,0,0.0,0,0.0,0,0.0
AÑO,0,0.0,0,0.0,0,0.0,0,0.0
MES,0,0.0,0,0.0,0,0.0,0,0.0
DIA,0,0.0,0,0.0,0,0.0,0,0.0
FRANJA_HORARIA,0,0.0,0,0.0,0,0.0,0,0.0
COMUNA,0,0.0,0,0.0,0,0.0,0,0.0
LONGITUD,0,0.0,0,0.0,0,0.0,0,0.0
LATITUD,0,0.0,0,0.0,0,0.0,0,0.0
VICTIMA,0,0.0,0,0.0,0,0.0,0,0.0


### b. DATAFRAME "VICTIMAS_LESIONES"...

#### (1) EXPLORACIÓN INICIAL...

In [25]:
victimas_lesiones.head(5)

,ID hecho,AAA,MM,DD,FECHA,VEHICULO_VICTIMA,SEXO,EDAD_VICTIMA,GRAVEDAD
0,LC-2019-0000053,2019,1,1,2019-01-01,sd,Varon,57,SD
1,LC-2019-0000063,2019,1,1,2019-01-01,sd,SD,SD,SD
2,LC-2019-0000079,2019,1,1,2019-01-01,sd,Varon,SD,SD
3,LC-2019-0000082,2019,1,1,2019-01-01,sd,Varon,45,SD
4,LC-2019-0000082,2019,1,1,2019-01-01,sd,Mujer,45,SD


In [26]:
victimas_lesiones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27605 entries, 0 to 27604
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   ID hecho          27605 non-null  object        
 1   AAA               27605 non-null  int64         
 2   MM                27605 non-null  int64         
 3   DD                27605 non-null  int64         
 4   FECHA             27605 non-null  datetime64[ns]
 5   VEHICULO_VICTIMA  27605 non-null  object        
 6   SEXO              27605 non-null  object        
 7   EDAD_VICTIMA      27605 non-null  object        
 8   GRAVEDAD          27605 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(5)
memory usage: 1.9+ MB


In [27]:
victimas_lesiones.dtypes

ID hecho                    object
AAA                          int64
MM                           int64
DD                           int64
FECHA               datetime64[ns]
VEHICULO_VICTIMA            object
SEXO                        object
EDAD_VICTIMA                object
GRAVEDAD                    object
dtype: object

In [28]:
victimas_lesiones.shape

(27605, 9)

In [29]:
victimas_lesiones.isnull().sum()

ID hecho            0
AAA                 0
MM                  0
DD                  0
FECHA               0
VEHICULO_VICTIMA    0
SEXO                0
EDAD_VICTIMA        0
GRAVEDAD            0
dtype: int64

In [30]:
victimas_lesiones.columns

Index(['ID hecho', 'AAA', 'MM', 'DD', 'FECHA ', 'VEHICULO_VICTIMA', 'SEXO',
       'EDAD_VICTIMA', 'GRAVEDAD'],
      dtype='object')

#### (2) PROCESO DE ETL DEL DATAFRAME...

In [31]:
# ELIMINACIÓN DE LAS COLUMNAS INNECESARIAS...

columnas_a_eliminar = ['FECHA ']
victimas_lesiones = victimas_lesiones.drop(columnas_a_eliminar, axis=1)

In [32]:
# RENOMBRAR LAS COLUMNAS...

nuevos_nombres = {'ID hecho':'ID_HECHO', 'AAA': 'AÑO', 'MM': 'MES', 'DD': 'DIA'}
victimas_lesiones = victimas_lesiones.rename(columns=nuevos_nombres)

In [33]:
# TRABAJO CON LA COLUMNA "VEHICULO_VICTIMA"...

victimas_lesiones['VEHICULO_VICTIMA'] = victimas_lesiones['VEHICULO_VICTIMA'].replace(['sd', 'SD'], 'MOTO')

In [34]:
# TRABAJO CON LA COLUMNA "SEXO"...

# Normalización de valores
valor_a_cambiar = ['varon', 'Varon', 'mujer', 'Mujer', 'Mujer ']
nuevo_valor = ['VARON', 'VARON', 'MUJER', 'MUJER', 'MUJER']
victimas_lesiones['SEXO'] = victimas_lesiones['SEXO'].replace(valor_a_cambiar, nuevo_valor)

# REemplazo de valores
victimas_lesiones['SEXO'] = victimas_lesiones['SEXO'].replace(['sd', 'SD'], 'VARON')

In [35]:
# TRABAJO CON LA COLUMNA "EDAD_VICTIMA"...

victimas_lesiones['EDAD_VICTIMA'] = victimas_lesiones['EDAD_VICTIMA'].replace(['sd', 'SD'], 29)

victimas_lesiones['EDAD_VICTIMA'] = victimas_lesiones['EDAD_VICTIMA'].apply(lambda x: int(x) if pd.notna(x) and isinstance(x, str) and x.isdigit() else x)

In [36]:
# TRABAJO CON LA COLUMNA "GRAVEDAD"...

# Normalización de valores
valor_a_cambiar = ['grave']
nuevo_valor = ['GRAVE']
victimas_lesiones['GRAVEDAD'] = victimas_lesiones['GRAVEDAD'].replace(valor_a_cambiar, nuevo_valor)

# REemplazo de valores
victimas_lesiones['GRAVEDAD'] = victimas_lesiones['GRAVEDAD'].replace(['sd', 'SD'], 'GRAVE')

In [37]:
# NORMALIZACIÓN DE LOS DATOS DE LAS COLUMNAS EN UN SOLO TIPO DE DATOS...

victimas_lesiones['VEHICULO_VICTIMA'] = victimas_lesiones['VEHICULO_VICTIMA'].astype(str)
victimas_lesiones['SEXO'] = victimas_lesiones['SEXO'].astype(str)
victimas_lesiones['GRAVEDAD'] = victimas_lesiones['GRAVEDAD'].astype(str)

In [38]:
# TRATAMIENTO DE VALORES DEL DATAFRAME...

# Identificar valores nulos en el DataFrame y contarlos por columna
valores_nulos = victimas_lesiones.isnull().sum()

# Identificar valores faltantes (en blanco) en el DataFrame y contarlos por columna
valores_faltantes = (victimas_lesiones == '').sum()

# Identificar valores iguales a "SD" en el DataFrame y contarlos por columna
valores_SD = (victimas_lesiones == 'SD').sum()

# Identificar valores iguales a "sd" en el DataFrame y contarlos por columna
valores_sd = (victimas_lesiones == 'sd').sum()

# Calcular porcentajes
porcentaje_nulos = (valores_nulos / len(victimas_lesiones)) * 100
porcentaje_faltantes = (valores_faltantes / len(victimas_lesiones)) * 100
porcentaje_SD = (valores_SD / len(victimas_lesiones)) * 100
porcentaje_sd = (valores_sd / len(victimas_lesiones)) * 100

# Crear un DataFrame con los resultados
resultados_datos_victimas_lesiones = pd.DataFrame({
    'Valores Nulos': valores_nulos,
    'Porcentaje Nulos': porcentaje_nulos,
    'Valores Faltantes': valores_faltantes,
    'Porcentaje Faltantes': porcentaje_faltantes,
    'Valores "SD"': valores_SD,
    'Porcentaje "SD"': porcentaje_SD,
    'Valores "sd"': valores_sd,
    'Porcentaje "sd"': porcentaje_sd
})

In [39]:
resultados_datos_victimas_lesiones

,Valores Nulos,Porcentaje Nulos,Valores Faltantes,Porcentaje Faltantes,"Valores ""SD""","Porcentaje ""SD""","Valores ""sd""","Porcentaje ""sd"""
ID_HECHO,0,0.0,0,0.0,0,0.0,0,0.0
AÑO,0,0.0,0,0.0,0,0.0,0,0.0
MES,0,0.0,0,0.0,0,0.0,0,0.0
DIA,0,0.0,0,0.0,0,0.0,0,0.0
VEHICULO_VICTIMA,0,0.0,0,0.0,0,0.0,0,0.0
SEXO,0,0.0,0,0.0,0,0.0,0,0.0
EDAD_VICTIMA,0,0.0,0,0.0,0,0.0,0,0.0
GRAVEDAD,0,0.0,0,0.0,0,0.0,0,0.0


## 5 - PROCESO DE ALMACENAMIENTO DE LOS DATOS LIMPIOS...

In [40]:
# Especifica la ruta del directorio donde deseas guardar los DataFrames en formato Parquet
carpeta_DATASET_LIMPIO = 'dataset_limpio'

# Verifica si la carpeta ya existe, si no, créala
if not os.path.exists(carpeta_DATASET_LIMPIO):
    os.makedirs(carpeta_DATASET_LIMPIO)

# Guarda los DataFrames en archivos Parquet
hechos_lesiones.to_parquet(os.path.join(carpeta_DATASET_LIMPIO, "hechos_lesiones.parquet"))
victimas_lesiones.to_parquet(os.path.join(carpeta_DATASET_LIMPIO, "victimas_lesiones.parquet"))